In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict,Annotated
from dotenv import load_dotenv
from pydantic import BaseModel,Field
import operator

In [2]:
load_dotenv()

True

In [14]:
model = ChatGroq(
     model="llama-3.1-8b-instant",
    temperature=0
)

In [15]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description = 'Detailed feedback for essay')
    score : int = Field(description='Score out of 10',ge=0,le=10)

    

In [16]:
structured_model = model.with_structured_output(EvaluationSchema)



In [17]:
essay = """Revolution of AI in India
The revolution of Artificial Intelligence (AI) in India marks one of the most transformative phases in the nation’s technological and economic journey. With its vast population, growing digital infrastructure, and strong talent base, India is uniquely positioned to harness AI for inclusive growth, innovation, and global leadership. From governance and healthcare to agriculture and education, AI is reshaping how India solves complex challenges and creates new opportunities.
The Rise of AI in India
India’s AI journey has been accelerated by rapid digitization, affordable internet access, and government-led initiatives. Programs such as Digital India have laid the foundation for large-scale data generation and digital service delivery. In parallel, India’s IT industry and startup ecosystem have adopted AI-driven solutions in areas like automation, analytics, and customer experience. The availability of vast datasets—through platforms such as Aadhaar—has further enabled the development of AI models tailored to India’s scale and diversity.
AI in Key Sectors
AI is playing a crucial role in transforming core sectors of the Indian economy:


Healthcare: AI-powered tools assist in early disease detection, medical imaging, and telemedicine, especially in rural and underserved regions. These technologies help bridge gaps in access and improve diagnostic accuracy.


Agriculture: Smart farming solutions use AI for crop monitoring, weather prediction, and soil analysis, helping farmers increase productivity and reduce losses.


Education: Adaptive learning platforms personalize education, while AI-driven assessment tools support teachers and students alike, improving learning outcomes across languages and regions.


Governance and Smart Cities: AI enhances traffic management, public safety, and service delivery in urban centers, supporting India’s vision of efficient and transparent governance.


Economic and Social Impact
The AI revolution is a powerful driver of economic growth. It boosts productivity, enables new business models, and positions India as a global hub for AI services and innovation. At the same time, AI supports social inclusion by improving access to healthcare, education, and financial services. When used responsibly, AI can reduce inequalities and empower marginalized communities.
Challenges and Ethical Considerations
Despite its promise, AI adoption in India faces challenges. Concerns around data privacy, algorithmic bias, job displacement, and digital divides must be addressed. Building robust ethical frameworks, investing in reskilling programs, and ensuring transparent AI governance are essential for sustainable progress.
The Road Ahead
India’s focus on “AI for All” highlights its commitment to inclusive and responsible innovation. With continued investment in research, education, and policy support, India can leverage AI not only to accelerate economic development but also to address pressing social challenges. The AI revolution in India is not just about technology—it is about shaping a smarter, fairer, and more resilient future.
Conclusion:
The revolution of AI in India represents a defining moment in the nation’s development story. By balancing innovation with ethics and inclusion, India can emerge as a global leader in AI, demonstrating how technology can serve humanity at scale."""

In [18]:
prompt = f"""
You must respond using the following JSON schema.

Schema:
- feedback: string (detailed feedback only)
- score: integer (0 to 10 only, no text)

Rules:
- Do NOT include the score inside feedback
- Do NOT add extra text
- Provide both fields

Essay:
{essay}
"""



In [19]:
response = structured_model.invoke(prompt)

feedback = response.feedback
score = response.score

print("Feedback:", feedback)
print("Score:", score)



APIConnectionError: Connection error.

In [13]:
structured_model.invoke(prompt)

ConnectError: [Errno 11001] getaddrinfo failed

In [23]:
class UPSCState(TypedDict):

    essay : str
    language_feedback : str
    analysis_feedback : str
    clarity_feedback : str
    overall_feedback : str

    individual_scores : Annotated[list[int] , operator.add]
    avg_score : str

In [52]:
def evaluate_language(state: UPSCState):

    essay = state["essay"] 

    prompt = f"""
        Evaulate the language quality of essay
        You must respond using the following JSON schema.

        Schema:
        - feedback: string (detailed feedback only)
        - score: integer (0 to 10 only, no text)

        Rules:
    - Do NOT include the score inside feedback
    - Do NOT add extra text
    - Provide both fields

    Essay:
    {state[essay]}
    """
    output = structured_model.invoke(prompt)

    return {'language_feedback':output.feedback,'individual_scores':[output.score]}

In [53]:
def evaluate_analysis(state: UPSCState):

    essay = state["essay"] 

    prompt = f"""
        Evaulate the depth of analysis of the essay
        You must respond using the following JSON schema.

        Schema:
        - feedback: string (detailed feedback only)
        - score: integer (0 to 10 only, no text)

        Rules:
    - Do NOT include the score inside feedback
    - Do NOT add extra text
    - Provide both fields

    Essay:
    {state[essay]}
    """
    output = structured_model.invoke(prompt)

    return {'analysis_feedback':output.feedback,'individual_scores':[output.score]}

In [54]:
def evaluate_thought(state: UPSCState):

    essay = state["essay"] 

    prompt = f"""
        Evaulate the clarity of thought of the essay
        You must respond using the following JSON schema.

        Schema:
        - feedback: string (detailed feedback only)
        - score: integer (0 to 10 only, no text)

        Rules:
    - Do NOT include the score inside feedback
    - Do NOT add extra text
    - Provide both fields

    Essay:
    {state[essay]}
    """
    output = structured_model.invoke(prompt)

    return {'clarity_feedback':output.feedback,'individual_scores':[output.score]}

In [55]:
def final_evaluation(state):

    prompt = (
        f"Based on the following feedbacks, create a summarized feedback:\n\n"
        f"Language feedback: {state['language_feedback']}\n"
        f"In-depth analysis feedback: {state['analysis_feedback']}\n"
        f"Clarity of thought feedback: {state['clarity_feedback']}"
    )

    overall_feedback = model.invoke(prompt)

    avg_score = sum(state['individual_score']) / len(state['individual_score'])

    return {
        "overall_feedback": overall_feedback.content,
        "average_score": avg_score
    }


In [56]:
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)


graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_thought')

graph.add_edge('evaluate_language','final_evaluation')
graph.add_edge('evaluate_analysis','final_evaluation')
graph.add_edge('evaluate_thought','final_evaluation')

graph.add_edge('final_evaluation',END)

workflow = graph.compile()

In [57]:
initial_state = {
    'essay' : essay
}

workflow.invoke(initial_state)


KeyError: 'Revolution of AI in India\nThe revolution of Artificial Intelligence (AI) in India marks one of the most transformative phases in the nation’s technological and economic journey. With its vast population, growing digital infrastructure, and strong talent base, India is uniquely positioned to harness AI for inclusive growth, innovation, and global leadership. From governance and healthcare to agriculture and education, AI is reshaping how India solves complex challenges and creates new opportunities.\nThe Rise of AI in India\nIndia’s AI journey has been accelerated by rapid digitization, affordable internet access, and government-led initiatives. Programs such as Digital India have laid the foundation for large-scale data generation and digital service delivery. In parallel, India’s IT industry and startup ecosystem have adopted AI-driven solutions in areas like automation, analytics, and customer experience. The availability of vast datasets—through platforms such as Aadhaar